In [1]:
import collections
import pandas as pd
import functools
from functools import lru_cache

In [3]:
with open('../inputs/input19.txt', 'r') as f:
    lines = f.readlines()
n = len(lines)
blueprints = []
for i in range(n):
    lines[i] = lines[i].strip()
    lines[i] = lines[i].split("costs ")
    temp = []
    lines[i][1] = lines[i][1].split(" ")
    temp.append(int(lines[i][1][0]))
    lines[i][2] = lines[i][2].split(" ")
    temp.append(int(lines[i][2][0]))
    lines[i][3] = lines[i][3].split(" ")
    temp.append([int(lines[i][3][0]), int(lines[i][3][3])])
    lines[i][4] = lines[i][4].split(" ")
    temp.append([int(lines[i][4][0]), int(lines[i][4][3])])
    blueprints.append(temp)
blueprints
#blueprints[i] := the (i+1)th blueprint
#blueprints[i][0] := the cost in ore of making an ore robot
#blueprints[i][1] := the cost in ore of making a clay robot
#blueprints[i][2][0] := the cost in ore of making an obsidian robot
#blueprints[i][2][1] := the cost in clay of making an obsidian robot
#blueprints[i][3][0] := the cost in ore of making a geode robot
#blueprints[i][3][1] := the cost in obsidian of making a geode robot

[[4, 4, [3, 14], [3, 8]],
 [3, 3, [2, 14], [3, 17]],
 [2, 2, [2, 20], [2, 14]],
 [4, 3, [3, 14], [4, 17]],
 [4, 4, [4, 20], [2, 8]],
 [2, 4, [4, 15], [2, 20]],
 [4, 4, [2, 14], [3, 17]],
 [3, 3, [2, 13], [3, 12]],
 [4, 4, [4, 11], [4, 12]],
 [2, 3, [3, 18], [2, 19]],
 [3, 4, [2, 14], [3, 14]],
 [4, 3, [4, 15], [4, 9]],
 [2, 3, [3, 13], [3, 15]],
 [2, 2, [2, 7], [2, 14]],
 [2, 4, [3, 17], [4, 20]],
 [3, 3, [2, 15], [3, 9]],
 [4, 3, [4, 16], [2, 15]],
 [4, 4, [3, 6], [2, 14]],
 [4, 3, [2, 13], [2, 10]],
 [3, 3, [2, 16], [2, 18]],
 [3, 3, [3, 8], [2, 12]],
 [3, 3, [3, 6], [2, 16]],
 [4, 3, [4, 18], [3, 13]],
 [4, 3, [2, 20], [2, 9]],
 [4, 4, [3, 5], [3, 18]],
 [4, 4, [4, 5], [2, 10]],
 [4, 4, [4, 8], [4, 14]],
 [4, 4, [2, 14], [4, 15]],
 [4, 4, [4, 20], [2, 12]],
 [3, 4, [4, 14], [4, 10]]]

In [11]:
#robots represents an input array [ore robots, clay robots, obsidian robots, geode robots]
#resources represents an input array [ore, clay, obsidian, geodes]
@lru_cache(maxsize=None)
def find_max_geodes(robots, resources, time_remaining, blueprint_num):
    max_geodes = resources[3]
    #Ran out of time
    if time_remaining == 0:
        return robots[3]
    #If we have enough robots to just keep making geode robots, do that and immediately return
    if robots[0]>=blueprints[blueprint_num][3][0] and robots[2]>=blueprints[blueprint_num][3][1] and resources[0]>=blueprints[blueprint_num][3][0] and resources[2]>=blueprints[blueprint_num][3][1]:
        temp_resources = (resources[0]+robots[0]-blueprints[blueprint_num][3][0], resources[1]+robots[1], resources[2]+robots[2]-blueprints[blueprint_num][3][1], resources[3]+robots[3])
        temp_robots = (robots[0], robots[1], robots[2], robots[3]+1)
        max_geodes = max(max_geodes, find_max_geodes(temp_robots, temp_resources, time_remaining-1, blueprint_num))
        return max_geodes
    #Choose to build a geode robot
    if resources[0]>=blueprints[blueprint_num][3][0] and resources[2]>=blueprints[blueprint_num][3][1]:
        temp_resources = (resources[0]+robots[0]-blueprints[blueprint_num][3][0], resources[1]+robots[1], resources[2]+robots[2]-blueprints[blueprint_num][3][1], resources[3]+robots[3])
        temp_robots = (robots[0], robots[1], robots[2], robots[3]+1)
        max_geodes = max(max_geodes, find_max_geodes(temp_robots, temp_resources, time_remaining-1, blueprint_num))
    #Choose to not build anything
    temp_resources = (resources[0]+robots[0], resources[1]+robots[1], resources[2]+robots[2], resources[3]+robots[3])
    max_geodes = max(max_geodes, find_max_geodes(robots, temp_resources, time_remaining-1, blueprint_num))
    #Choose to build an ore robot
    max_ore_cost = max(blueprints[blueprint_num][0], blueprints[blueprint_num][1], blueprints[blueprint_num][2][0], blueprints[blueprint_num][3][0])
    if robots[0]<max_ore_cost and resources[0]>=blueprints[blueprint_num][0]:
        temp_resources = (resources[0]+robots[0]-blueprints[blueprint_num][0], resources[1]+robots[1], resources[2]+robots[2], resources[3]+robots[3])
        temp_robots = (robots[0]+1, robots[1], robots[2], robots[3])
        max_geodes = max(max_geodes, find_max_geodes(temp_robots, temp_resources, time_remaining-1, blueprint_num))
    #Choose to build a clay robot
    max_clay_cost = blueprints[blueprint_num][2][1]
    if robots[1]<max_clay_cost and resources[0]>=blueprints[blueprint_num][1]:
        temp_resources = (resources[0]+robots[0]-blueprints[blueprint_num][1], resources[1]+robots[1], resources[2]+robots[2], resources[3]+robots[3])
        temp_robots = (robots[0], robots[1]+1, robots[2], robots[3])
        max_geodes = max(max_geodes, find_max_geodes(temp_robots, temp_resources, time_remaining-1, blueprint_num))
    #Choose to build an obsidian robot
    max_obsidian_cost = blueprints[blueprint_num][3][1]
    if robots[2]<max_obsidian_cost and resources[0]>=blueprints[blueprint_num][2][0] and resources[1]>=blueprints[blueprint_num][2][1]:
        temp_resources = (resources[0]+robots[0]-blueprints[blueprint_num][2][0], resources[1]+robots[1]-blueprints[blueprint_num][2][1], resources[2]+robots[2], resources[3]+robots[3])
        temp_robots = (robots[0], robots[1], robots[2]+1, robots[3])
        max_geodes = max(max_geodes, find_max_geodes(temp_robots, temp_resources, time_remaining-1, blueprint_num))
    return max_geodes

In [5]:
#This for-loop kept crashing at the same blueprint, so I decided to manually print every single blueprint's result and then store it in an array
results = []
for i in range(len(blueprints)):
    print(find_max_geodes((1,0,0,0),(0,0,0,0),25, i))
    results.append(find_max_geodes((1,0,0,0),(0,0,0,0),25, i))
results

2
1
3
1
1
1
0
3
1
1
1
2
5


KeyboardInterrupt: 

In [ ]:
print(find_max_geodes((1,0,0,0),(0,0,0,0),25, 0))

In [9]:
res = [2,1,3,1,1,1,0,3,1,1,1,2,5,'?',1,4,1,3,3,1,7,7,1,1,3,5,1,1,0,2]
len(res)

30

In [46]:
for i in range(len(results)):
    results[i] = (i+1)*results[i]
print(sum(results))

33
